This notebook is part of Andreu's (esdandreu@gmail.com) Master Thesis work at
Keio University.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AcousticOdometry/AO/blob/main/notebooks/models.ipynb)


# Setup

This section will take care of installing the necessary packages as well as
configuring some environment variables.


## Colab

Assess wether the notebook is being executed in [Google
Colab](https://colab.research.google.com/) and if so, set up the software
needed in Colab runtime.


In [1]:
%%capture
try:
    from google import colab
    COLAB_RUNTIME = True
    %pip install torchinfo
    colab.drive.mount('/content/drive')
except ImportError:
    COLAB_RUNTIME = False

## Packages


In [2]:
import logging
import numpy as np

from pathlib import Path
from datetime import datetime
from tqdm.notebook import tqdm
from typing import Dict, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchinfo import summary
from torchvision import datasets, transforms

### AO

Setup Acoustic Odometry python package. If this notebook is being executed in
[Colab](#colab), the package will be installed from Github. Because of this, a
Github [personal access
token](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token)
is needed for the installation.

If the notebook is not running on Colab and the package is not already
installed, installation instructions will be prompted.


In [3]:
if COLAB_RUNTIME:
    import subprocess
    import requests
    import sys
    import os
    #@markdown Use a [GitHub Personal Access Token](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token)
    GITHUB_TOKEN = ''  #@param {type:"string"}
    auth = requests.auth.HTTPBasicAuth('', GITHUB_TOKEN)
    response = requests.get(
        "https://api.github.com/repos/AcousticOdometry/AO/releases/latest",
        auth=auth
        )
    try:
        response.raise_for_status()
    except requests.HTTPError as e:
        raise RuntimeError(
            'Check GITHUB_TOKEN is a Personal Access Token with repo access'
            )
    headers = {'Accept': 'application/octet-stream'}
    for asset in response.json()['assets']:
        r = requests.get(
            asset['url'], auth=auth, allow_redirects=True, headers=headers
            )
        r.raise_for_status()
        wheel_name = asset['name']
        with open(wheel_name, 'wb') as f:
            f.write(r.content)
        try:
            result = subprocess.check_output([
                sys.executable, '-m', 'pip', 'install', wheel_name
                ])
            print(f'Installed {wheel_name}')
            break
        except subprocess.CalledProcessError as e:
            pass
        finally:
            os.remove(wheel_name)
    import ao
else:
    try:
        import ao
    except ImportError:
        raise ImportError(
            "Acoustic Odometry python extension is not installed. Check "
            r"https://github.com/AcousticOdometry/AO#readme"
            " for detailed instructions."
            )

## Configure logging

In [4]:
logging_folder = Path().parent / 'logs'
logging_folder.mkdir(exist_ok=True)
stream_handler = logging.StreamHandler()
stream_handler.setLevel(logging.INFO)
logging.basicConfig(
    format="[%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler(
            logging_folder / datetime.now().strftime('%Y%m%d_%H%M%S.log')
            ),
        stream_handler,
        ],
    level=logging.DEBUG
    )


# Models

## Model definition

In [5]:
from abc import abstractmethod

class AcousticOdometryModel(nn.Module):

    @abstractmethod
    def forward(self, x):
        pass

# Reset the subclasses to allow changes without restarting the kernel
for subclass in AcousticOdometryModel.__subclasses__():
    del subclass

In [6]:
class CNNet(AcousticOdometryModel):
    def __init__(self, classes: int):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=5)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(210816, 512)
        self.fc2 = nn.Linear(512, classes)


    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        #x = x.view(x.size(0), -1)
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        return F.log_softmax(x,dim=1)  

In [7]:
AcousticOdometryModel.__subclasses__()

[__main__.CNNet]

## Load data

In [8]:
if COLAB_RUNTIME:
    # @markdown Check where is the experiment folder situated in your drive folder.
    # @markdown Remember that if you have been shared the folder, you can
    # @markdown [add a shortcut to your drive](https://support.google.com/drive/answer/9700156?hl=en&co=GENIE.Platform%3DDesktop)
    # @markdown in order to make it available in google colab.
    experiment = "/content/drive/MyDrive/VAO_WheelTestBed-Experiment-1"  #@param {type:"string"}
    EXPERIMENT_FOLDER = Path(experiment)
    if not EXPERIMENT_FOLDER.is_dir():
        raise RuntimeError(f'Invalid experiment folder {EXPERIMENT_FOLDER}')
else:
    EXPERIMENT_FOLDER = ao.dataset.utils.get_folder(
        env='WHEELTESTBED_EXPERIMENT1'
        )
DATASETS_FOLDER = EXPERIMENT_FOLDER / 'datasets'
MODELS_FOLDER = EXPERIMENT_FOLDER / 'models'
MODELS_FOLDER.mkdir(exist_ok=True)

In [9]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(DEVICE))

[INFO] Using cuda device


In [10]:
def load_dataset(
    dataset_folder: Path,
    train_split: float = 0.8,
    batch_size: int = 15,
    ) -> Tuple[Dict[str, torch.utils.data.DataLoader], Dict[str, int]]:
    dataset = datasets.DatasetFolder(
        root=dataset_folder,
        loader=np.load,
        extensions=['.npy'],
        transform=transforms.Compose([
            transforms.ToTensor(),
            ])
        )
    train_size = int(train_split * len(dataset))
    test_size = len(dataset) - train_size
    loaders = {}
    for split, name in zip(
        torch.utils.data.random_split(dataset, [train_size, test_size]),
        ['train', 'test']
        ):
        loaders[name] = torch.utils.data.DataLoader(
            split,
            batch_size=batch_size,
            num_workers=2 if DEVICE == 'cuda' else 1,
            shuffle=True,
            pin_memory=True if DEVICE == 'cuda' else False,
            )
        logging.info(
            f'{name} set: {len(split)} samples, {len(loaders[name])} batches'
            )
    return loaders, dataset.class_to_idx


## Train the model

In [11]:
def train_epoch(
    dataloader: torch.utils.data.DataLoader,
    model: torch.nn.Module,
    cost_function: torch.nn.modules.loss._Loss,
    optimizer: torch.optim.Optimizer,
    ):
    model.train()
    size = len(dataloader.dataset)
    # TODO use tqdm
    for batch_n, (X, Y) in enumerate(dataloader):
        X, Y = X.float().to(DEVICE), Y.to(DEVICE)
        optimizer.zero_grad()
        prediction = model(X)
        loss = cost_function(prediction, Y)
        loss.backward()
        optimizer.step()

        if batch_n % 100 == 0:
            loss, current = loss.item(), batch_n * len(X)
            logging.debug(f'loss: {loss:>7f}  [{current:>5d}/{size:>5d}]')


def test(
    dataloader: torch.utils.data.DataLoader,
    model: torch.nn.Module,
    cost_function: torch.nn.modules.loss._Loss,
    ):
    model.eval()
    size = len(dataloader.dataset)

    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, Y in dataloader:
            X, Y = X.float().to(DEVICE), Y.to(DEVICE)
            pred = model(X)

            test_loss += cost_function(pred, Y).item()
            correct += (pred.argmax(1) == Y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    logging.info(f'acc: {(100*correct):>0.1f}%, avg loss: {test_loss:>8f}')


In [14]:
def train(
    dataset_folder: Path,
    model_class: AcousticOdometryModel,
    epochs: int = 15,
    cost_function: torch.nn.modules.loss._Loss = torch.nn.CrossEntropyLoss(),
    optimizer_class: torch.optim.Optimizer = torch.optim.Adam,
    optimizer_options: dict = {'lr': 0.0001},
    ):
    loaders, class_to_idx = load_dataset(dataset_folder)
    # ! A bit sketchy, in the future the classes should be fixed
    model = model_class(classes=len(class_to_idx)).to(DEVICE)
    optimizer = optimizer_class(model.parameters(), **optimizer_options)
    for t in range(epochs):
        logging.info(f'Epoch {t}')
        train_epoch(loaders['train'], model, cost_function, optimizer)
        test(loaders['test'], model, cost_function)
    return model

In [15]:
dataset_name = 'numpy-arrays'
# TODO config logging
model = train(
    dataset_folder=DATASETS_FOLDER / dataset_name,
    model_class=CNNet,
    epochs=15,
    optimizer_options={'lr': 0.0001},
    )
torch.save(
    model, MODELS_FOLDER / (
        f"name_{dataset_name.replace('_','-')};" +
        f"{datetime.now().strftime('date_%Y-%m-%d;time_%H-%M-%S')}.pt"
        )
    )
# TODO close logging and copy to MODELS FOLDER
summary(model, input_size=(15, 1, 256, 120))

[INFO] train set: 32158 samples, 2144 batches
[INFO] test set: 8040 samples, 536 batches
[INFO] Epoch 0
[INFO] acc: 27.0%, avg loss: 0.104499
[INFO] Epoch 1
[INFO] acc: 49.0%, avg loss: 0.077143
[INFO] Epoch 2
[INFO] acc: 78.3%, avg loss: 0.042562
[INFO] Epoch 3
[INFO] acc: 81.2%, avg loss: 0.034188
[INFO] Epoch 4
[INFO] acc: 83.5%, avg loss: 0.029783
[INFO] Epoch 5
[INFO] acc: 84.6%, avg loss: 0.028051
[INFO] Epoch 6
[INFO] acc: 85.2%, avg loss: 0.026831
[INFO] Epoch 7
[INFO] acc: 85.3%, avg loss: 0.025986
[INFO] Epoch 8
[INFO] acc: 86.5%, avg loss: 0.024434
[INFO] Epoch 9
[INFO] acc: 92.3%, avg loss: 0.017815
[INFO] Epoch 10
[INFO] acc: 92.2%, avg loss: 0.016778
[INFO] Epoch 11
[INFO] acc: 92.9%, avg loss: 0.015925
[INFO] Epoch 12
[INFO] acc: 92.8%, avg loss: 0.015595
[INFO] Epoch 13
[INFO] acc: 93.1%, avg loss: 0.015481
[INFO] Epoch 14
[INFO] acc: 93.0%, avg loss: 0.016040


Layer (type:depth-idx)                   Output Shape              Param #
CNNet                                    --                        --
├─Conv2d: 1-1                            [15, 64, 252, 116]        1,664
├─Conv2d: 1-2                            [15, 128, 122, 54]        204,928
├─Dropout2d: 1-3                         [15, 128, 122, 54]        --
├─Flatten: 1-4                           [15, 210816]              --
├─Linear: 1-5                            [15, 512]                 107,938,304
├─Linear: 1-6                            [15, 7]                   3,591
Total params: 108,148,487
Trainable params: 108,148,487
Non-trainable params: 0
Total mult-adds (G): 22.60
Input size (MB): 1.84
Forward/backward pass size (MB): 325.76
Params size (MB): 432.59
Estimated Total Size (MB): 760.19